In [1]:
import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as gbr
from datetime import datetime,timedelta

In [2]:
df0 = pd.read_csv('data_CA_1.csv')
df1 = pd.read_csv('data_CA_2.csv')
df2 = pd.read_csv('data_CA_3.csv')
df3 = pd.read_csv('data_CA_4.csv')
df4 = pd.read_csv('data_TX_1.csv')
df5 = pd.read_csv('data_TX_2.csv')
df6 = pd.read_csv('data_TX_3.csv')
df7 = pd.read_csv('data_WI_1.csv')
df8 = pd.read_csv('data_WI_2.csv')
df9 = pd.read_csv('data_WI_3.csv')

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def preprocess(df):
    df.drop(columns = ['Unnamed: 0','store_id','id','weekday','sell_price','event_name_1','event_name_2','event_type_2'],inplace=True)
    
    
    # get the day of month
    date = pd.to_datetime(df['date'],format= '%Y-%m-%d')
    mday = []
    for x in date:
        mday.append(x.strftime("%d"))
    df['mday'] = mday
    df.mday = df.mday.astype(int)
    # get the day number
    df['day'] = df.d.apply(lambda x: x[2:])
    df.day = df.day.astype(int)
    # get the week number
    df['week'] = df.wm_yr_wk.apply(lambda x: x%100)
    
    # dummy to denote national events
    df['national'] = np.where(df.event_type_1=='National',1,0)
    
    df['snap_CA'] = df.snap.apply(lambda x: x[0])
    df['snap_TX'] = df.snap.apply(lambda x: x[2])
    df['snap_WI'] = df.snap.apply(lambda x: x[4])
    
    df.drop(columns = ['d','wm_yr_wk','event_type_1','date','snap'],inplace=True)
    
    return df

In [ ]:
df = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
dfs = []
for i in range(10):
    dfs.append(preprocess(df[i]))

In [37]:
def clean(df):

    df_y = df.groupby('day').sum().demand.reset_index(drop = True)
    df_x = df.drop(columns = ['demand']).drop_duplicates().reset_index(drop = True)
    
    df_x['demand_28'] = df_y.shift(28)
    
    df_x = df_x.dropna().reset_index(drop=True)
    df_y = df_y[28:].reset_index(drop=True)

    return df_x,df_y


In [4]:
def split(df_x,df_y,num):
    # train-valid split, use last 28 days as validation
    df_xtrain = df_x.iloc[num:-28,:]
    df_xvalid = df_x.iloc[-28:,:]
    df_ytrain = df_y.iloc[num:-28]
    df_yvalid = df_y.iloc[-28:]
    
    return df_xtrain,df_ytrain,df_xvalid,df_yvalid

In [5]:
def rmsse(y_train,y_test,y_pred):
    y_train1 = y_train.shift()
    n = len(y_train)
    h = len(y_test)
    dividor = sum((y_train[1:]-y_train1[1:])**2)/(n-1)
    divident = sum((y_test-y_pred)**2)/h
    
    return divident/dividor

In [6]:
def rf_select(df_xtrain,df_ytrain,df_xvalid,df_yvalid,m,n):
    model = rfr(random_state=1,n_jobs=-1,max_features=m,n_estimators=n).fit(df_xtrain,df_ytrain)
    df_ypred = model.predict(df_xvalid)

    return rmsse(df_ytrain,df_yvalid,df_ypred)

In [13]:
weight = np.array([0.110888286,0.110246513,0.155628193,0.065608342,0.0775605,0.09520139,0.096534717,0.087117424,0.116621569,0.084593065])


In [8]:
error_table = pd.DataFrame(index=range(1,12),columns = [100,150,200,250,300,350,400])

In [20]:
# dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
stores = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
for m in range(1,12):
    for n in [100,150,200,250,300,350,400]:
        a = []
        Loss = np.array([0])
        for i in range(len(dfs)):
            df_x,df_y = clean(dfs[i])
            df_xtrain,df_ytrain,df_xvalid,df_yvalid = split(df_x,df_y,-759)
            loss= rf_select(df_xtrain,df_ytrain,df_xvalid,df_yvalid,m=m,n=n)
            a.append(loss)
            Loss = np.array(a)
        error_table.loc[m,n] = sum(Loss*weight)

In [24]:
error_table

,100,150,200,250,300,350,400
1,0.511987,0.497938,0.473267,0.481701,0.479477,0.482138,0.482827
2,0.361149,0.353028,0.350271,0.354369,0.352122,0.355914,0.35175
3,0.316011,0.31065,0.307865,0.304425,0.30027,0.300153,0.303828
4,0.298855,0.291995,0.292824,0.288443,0.28563,0.286286,0.290502
5,0.262247,0.26734,0.265132,0.267927,0.269266,0.266978,0.2701
6,0.262776,0.261621,0.255362,0.264801,0.267348,0.26701,0.271002
7,0.265625,0.273385,0.268444,0.266063,0.268401,0.270194,0.274057
8,0.270708,0.272619,0.269438,0.268092,0.268685,0.271118,0.272595
9,0.274172,0.278531,0.286543,0.286697,0.282279,0.285953,0.287031
10,0.279488,0.279546,0.279526,0.28086,0.28563,0.285848,0.286596


In [26]:
df_x.columns

Index(['wday', 'month', 'year', 'mday', 'day', 'week', 'national', 'snap_CA',
       'snap_TX', 'snap_WI', 'demand_28'],
      dtype='object')

In [30]:
valid_train = pd.DataFrame(columns = ['wday', 'month', 'year', 'mday', 'day', 'week', 'national', 'snap_CA','snap_TX', 'snap_WI', 'demand_28'])

date = []
for i in range(1,57):
    date.append(pd.to_datetime('2016-04-24')+timedelta(days=i))
wday = []
for x in date:
    num = int(x.strftime("%w"))
    if num == 5:
        wday.append(7)
    else:
        wday.append((num+2)%7)
valid_train['wday'] = wday

month = []
for x in date:
    month.append(int(x.strftime("%m")))
valid_train['month'] = month

valid_train['year'] = [2016]*56

mday = []
for x in date:
    mday.append(int(x.strftime("%d")))
valid_train['mday'] = mday

valid_train['day'] = range(1914,1914+56)

week = [13]*5+[14]*7+[15]*7+[16]*7+[17]*7+[18]*7+[19]*7+[20]*7+[21]*2
valid_train['week'] = week

# only 5/30/2016 is a federal holiday
national = [0]*36+[1]+[0]*19
valid_train['national'] = national

mask1 = (valid_train.mday<=10)
valid_train['snap_CA']=np.where(mask1,1,0)

mask2 = (valid_train.mday==1)|(valid_train.mday==3)|(valid_train.mday==5)|(valid_train.mday==6)|(valid_train.mday==7)|(valid_train.mday==9)|(valid_train.mday==11)|(valid_train.mday==12)|(valid_train.mday==13)|(valid_train.mday==15)
valid_train['snap_TX']=np.where(mask2,1,0)

mask3 = (valid_train.mday==2)|(valid_train.mday==3)|(valid_train.mday==5)|(valid_train.mday==6)|(valid_train.mday==8)|(valid_train.mday==9)|(valid_train.mday==11)|(valid_train.mday==12)|(valid_train.mday==14)|(valid_train.mday==15)
valid_train['snap_WI']=np.where(mask3,1,0)

valid_train['demand_28'] = [0]*56

In [38]:
def rf_forecast(df_x,df_y,valid_train):
    model = rfr(random_state=1,n_jobs=-1,max_features=5,n_estimators=100).fit(df_x,df_y)
    valid_train.demand_28[:28] = df_y[-28:]
    df_ypred1 = model.predict(valid_train.iloc[:28,:])
    valid_train.demand_28[28:] = df_ypred1
    df_ypred = model.predict(valid_train)

    return df_ypred

In [33]:
pred = pd.DataFrame(columns=['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3'])

In [39]:
dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
stores = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
for i in range(len(dfs)):
    df_x,df_y = clean(dfs[i])
    df_ypred = rf_forecast(df_x,df_y,valid_train)
    pred[f'{stores[i]}'] = df_ypred

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [40]:
pred

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,4132.22,3790.46,5782.02,2572.05,2914.47,3609.35,3473.05,3012.03,4524.20,3094.91
1,3724.43,3811.81,5477.25,2390.28,2699.84,3128.85,3350.68,2903.99,4402.39,3260.95
2,3650.26,3688.18,5343.66,2401.86,2840.04,3447.71,3309.50,2832.76,4284.95,3033.02
3,3697.90,3648.12,5280.06,2430.92,2698.71,3412.15,3238.69,2841.62,4366.39,3060.25
4,4350.15,4461.35,5511.47,2490.39,2975.66,3804.91,3483.73,3937.45,4659.71,3552.16
5,5530.99,6047.56,7389.45,2967.83,3559.71,4335.66,3992.65,4809.02,5095.72,4337.59
6,6016.54,6022.62,7975.52,3053.87,4129.60,4810.12,4482.86,4791.40,5828.77,4611.86
7,4955.26,4041.82,6736.70,2752.72,3011.93,3665.27,3699.11,3357.80,5913.32,3915.58
8,4016.64,3763.75,6369.05,2550.36,3089.36,3741.71,3840.89,3352.75,5939.72,3961.37
9,3952.02,3769.04,5750.66,2495.80,3022.78,3494.13,3554.04,3179.77,5331.37,3504.13


In [41]:
pred = pred.transpose()

In [42]:
pred.to_csv('store_pred.csv')

In [5]:
# pred = pd.read_csv('store_pred.csv')

In [10]:
pred

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,4132.22,3724.43,3650.26,3697.90,4350.15,5530.99,6016.54,4955.26,4016.64,3952.02,...,4638.55,5629.33,5762.07,4441.61,3776.81,3732.35,3797.73,4527.59,5518.43,5655.31
1,3790.46,3811.81,3688.18,3648.12,4461.35,6047.56,6022.62,4041.82,3763.75,3769.04,...,4392.85,5618.11,5610.13,3876.84,3724.52,3711.24,3601.30,4384.01,5769.35,5644.04
2,5782.02,5477.25,5343.66,5280.06,5511.47,7389.45,7975.52,6736.70,6369.05,5750.66,...,6183.26,7454.72,8106.50,6285.53,6031.30,5809.18,5768.45,6118.91,7322.65,7703.15
3,2572.05,2390.28,2401.86,2430.92,2490.39,2967.83,3053.87,2752.72,2550.36,2495.80,...,2495.98,2961.79,2999.43,2721.66,2473.32,2454.82,2465.97,2515.58,2975.41,3021.38
4,2914.47,2699.84,2840.04,2698.71,2975.66,3559.71,4129.60,3011.93,3089.36,3022.78,...,3085.81,3893.63,4138.20,3408.28,3006.16,3457.82,2953.31,3081.96,3813.39,4134.42
5,3609.35,3128.85,3447.71,3412.15,3804.91,4335.66,4810.12,3665.27,3741.71,3494.13,...,3834.13,4700.28,4832.98,4037.36,3561.07,3804.03,3552.84,3951.23,4598.89,4680.32
6,3473.05,3350.68,3309.50,3238.69,3483.73,3992.65,4482.86,3699.11,3840.89,3554.04,...,3613.51,4436.36,4500.54,4060.75,3512.65,3758.69,3430.06,3611.08,4247.92,4343.26
7,3012.03,2903.99,2832.76,2841.62,3937.45,4809.02,4791.40,3357.80,3352.75,3179.77,...,3926.49,4955.01,4800.96,3188.98,3204.29,3222.30,3163.79,4004.25,4798.78,4556.58
8,4524.20,4402.39,4284.95,4366.39,4659.71,5095.72,5828.77,5913.32,5939.72,5331.37,...,5292.51,7085.65,7073.75,4837.04,5499.88,5805.15,4687.95,4747.10,5395.92,5285.57
9,3094.91,3260.95,3033.02,3060.25,3552.16,4337.59,4611.86,3915.58,3961.37,3504.13,...,3725.47,5156.73,4918.90,3580.74,3755.09,4077.63,3354.70,3781.83,4583.50,4384.77


In [6]:
df0 = pd.read_csv('data_CA_1.csv')
df1 = pd.read_csv('data_CA_2.csv')
df2 = pd.read_csv('data_CA_3.csv')
df3 = pd.read_csv('data_CA_4.csv')
df4 = pd.read_csv('data_TX_1.csv')
df5 = pd.read_csv('data_TX_2.csv')
df6 = pd.read_csv('data_TX_3.csv')
df7 = pd.read_csv('data_WI_1.csv')
df8 = pd.read_csv('data_WI_2.csv')
df9 = pd.read_csv('data_WI_3.csv')

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def get_ratio(df):
    df['day'] = df.d.apply(lambda x: x[2:])
    df.day = df.day.astype(int)
    last = df[df.day>1913-28]
    sum_demand = sum(last.demand)
    sep_demand = last.groupby(['id']).sum().demand
    
    sep_demand['ratio'] = sep_demand/sum_demand
        
    return sep_demand['ratio']

In [8]:
def forecast(df,store_name):
    result = pd.DataFrame(columns=range(1,57))
    ratio = get_ratio(df)
    for x in range(1,57):
        result[x]=ratio*pred.loc[store_name,:][x-1]
    return result

In [11]:
# pred.index = stores

In [13]:
result = pd.DataFrame(columns=range(1,57))

dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
stores = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

for i in range(len(dfs)):
    sub_data = forecast(dfs[i],stores[i])
    result = pd.concat([result,sub_data])

In [14]:
result

,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,54,55,56
FOODS_1_001_CA_1,1.065713,0.960542,0.941414,0.953700,1.121917,1.426460,1.551685,1.277977,1.035904,1.019238,...,1.196297,1.451822,1.486056,1.145505,0.974051,0.962585,0.979447,1.167680,1.423221,1.458522
FOODS_1_002_CA_1,0.387532,0.349288,0.342332,0.346800,0.407970,0.518713,0.564249,0.464719,0.376692,0.370632,...,0.435017,0.527935,0.540384,0.416547,0.354200,0.350031,0.356162,0.424611,0.517535,0.530372
FOODS_1_003_CA_1,0.710475,0.640362,0.627609,0.635800,0.747945,0.950973,1.034456,0.851985,0.690603,0.679492,...,0.797531,0.967881,0.990704,0.763670,0.649368,0.641723,0.652964,0.778453,0.948814,0.972348
FOODS_1_004_CA_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_1_005_CA_1,1.098007,0.989650,0.969941,0.982600,1.155915,1.469686,1.598705,1.316704,1.067295,1.050124,...,1.232548,1.495817,1.531088,1.180218,1.003568,0.991754,1.009127,1.203064,1.466348,1.502720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3,0.166080,0.174991,0.162759,0.164221,0.190618,0.232766,0.247484,0.210120,0.212577,0.188040,...,0.199918,0.276723,0.263960,0.192151,0.201507,0.218816,0.180021,0.202942,0.245962,0.235298
HOUSEHOLD_2_513_WI_3,0.249121,0.262486,0.244139,0.246331,0.285926,0.349149,0.371226,0.315180,0.318865,0.282060,...,0.299877,0.415084,0.395940,0.288227,0.302261,0.328224,0.270032,0.304413,0.368943,0.352946
HOUSEHOLD_2_514_WI_3,0.138400,0.145826,0.135633,0.136850,0.158848,0.193971,0.206236,0.175100,0.177147,0.156700,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081
HOUSEHOLD_2_515_WI_3,0.138400,0.145826,0.135633,0.136850,0.158848,0.193971,0.206236,0.175100,0.177147,0.156700,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081


In [15]:
result.to_csv('result.csv')

In [16]:
validation = result.loc[:,:28]
evaluation = result.loc[:,29:]

In [17]:
validation.reset_index(inplace=True)
for i in range(30490):
    validation['index'][i] = validation['index'][i]+'_validation'
validation.rename(columns = {'index':'id'},inplace = True)
validation.set_index('id',inplace=True)

evaluation.reset_index(inplace=True)
for i in range(30490):
    evaluation['index'][i] = evaluation['index'][i]+'_evaluation'
evaluation.rename(columns = {'index':'id'},inplace=True)
evaluation.set_index('id',inplace=True)

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [18]:
columns = []
for i in range(1,29):
    columns.append(f'F{i}')
validation.columns = columns
evaluation.columns = columns
result=pd.concat([validation,evaluation])

In [19]:
result

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,1.065713,0.960542,0.941414,0.953700,1.121917,1.426460,1.551685,1.277977,1.035904,1.019238,...,1.135594,1.449741,1.467887,1.137469,0.953009,0.947763,0.951119,1.150975,1.441176,1.453148
FOODS_1_002_CA_1_validation,0.387532,0.349288,0.342332,0.346800,0.407970,0.518713,0.564249,0.464719,0.376692,0.370632,...,0.412943,0.527178,0.533777,0.413625,0.346549,0.344641,0.345861,0.418537,0.524064,0.528417
FOODS_1_003_CA_1_validation,0.710475,0.640362,0.627609,0.635800,0.747945,0.950973,1.034456,0.851985,0.690603,0.679492,...,0.757063,0.966494,0.978591,0.758313,0.635339,0.631842,0.634079,0.767317,0.960784,0.968765
FOODS_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_1_005_CA_1_validation,1.098007,0.989650,0.969941,0.982600,1.155915,1.469686,1.598705,1.316704,1.067295,1.050124,...,1.170006,1.493672,1.512368,1.171938,0.981888,0.976483,0.979940,1.185853,1.484848,1.497182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3_evaluation,0.167236,0.168384,0.163147,0.163437,0.195461,0.220885,0.220797,0.164807,0.159563,0.182973,...,0.199918,0.276723,0.263960,0.192151,0.201507,0.218816,0.180021,0.202942,0.245962,0.235298
HOUSEHOLD_2_513_WI_3_evaluation,0.250855,0.252576,0.244720,0.245155,0.293192,0.331328,0.331196,0.247211,0.239344,0.274459,...,0.299877,0.415084,0.395940,0.288227,0.302261,0.328224,0.270032,0.304413,0.368943,0.352946
HOUSEHOLD_2_514_WI_3_evaluation,0.139364,0.140320,0.135956,0.136197,0.162884,0.184071,0.183998,0.137339,0.132969,0.152477,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081


In [20]:
result.to_csv('final_result.csv')

In [21]:
pd.read_csv('final_result.csv')

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.065713,0.960542,0.941414,0.953700,1.121917,1.426460,1.551685,1.277977,1.035904,...,1.135594,1.449741,1.467887,1.137469,0.953009,0.947763,0.951119,1.150975,1.441176,1.453148
1,FOODS_1_002_CA_1_validation,0.387532,0.349288,0.342332,0.346800,0.407970,0.518713,0.564249,0.464719,0.376692,...,0.412943,0.527178,0.533777,0.413625,0.346549,0.344641,0.345861,0.418537,0.524064,0.528417
2,FOODS_1_003_CA_1_validation,0.710475,0.640362,0.627609,0.635800,0.747945,0.950973,1.034456,0.851985,0.690603,...,0.757063,0.966494,0.978591,0.758313,0.635339,0.631842,0.634079,0.767317,0.960784,0.968765
3,FOODS_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,FOODS_1_005_CA_1_validation,1.098007,0.989650,0.969941,0.982600,1.155915,1.469686,1.598705,1.316704,1.067295,...,1.170006,1.493672,1.512368,1.171938,0.981888,0.976483,0.979940,1.185853,1.484848,1.497182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_512_WI_3_evaluation,0.167236,0.168384,0.163147,0.163437,0.195461,0.220885,0.220797,0.164807,0.159563,...,0.199918,0.276723,0.263960,0.192151,0.201507,0.218816,0.180021,0.202942,0.245962,0.235298
60976,HOUSEHOLD_2_513_WI_3_evaluation,0.250855,0.252576,0.244720,0.245155,0.293192,0.331328,0.331196,0.247211,0.239344,...,0.299877,0.415084,0.395940,0.288227,0.302261,0.328224,0.270032,0.304413,0.368943,0.352946
60977,HOUSEHOLD_2_514_WI_3_evaluation,0.139364,0.140320,0.135956,0.136197,0.162884,0.184071,0.183998,0.137339,0.132969,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081
60978,HOUSEHOLD_2_515_WI_3_evaluation,0.139364,0.140320,0.135956,0.136197,0.162884,0.184071,0.183998,0.137339,0.132969,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081
